In [6]:
%load_ext autoreload
%autoreload 2

import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from tasks import RoadTypeClfTask, TravelTimeEstimation
from sklearn import metrics, linear_model
from evaluation import Evaluation
from generator import RoadNetwork
import torch
from models import GAEModel, Node2VecModel, Toast
import numpy as np
import networkx as nx

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
network = RoadNetwork()
network.load("../osm_data/porto")
data = network.generate_road_segment_pyg_dataset()

In [43]:
decoder = linear_model.LogisticRegression(multi_class="multinomial", max_iter=1000)
y = np.array([network.gdf_edges.loc[n]["highway_enc"] for n in network.line_graph.nodes])
roadclf = RoadTypeClfTask(decoder, y)
roadclf.register_metric(name="f1_micro", metric_func=metrics.f1_score, args={"average": "micro"})
roadclf.register_metric(name="f1_macro", metric_func=metrics.f1_score, args={"average": "macro"})
roadclf.register_metric(name="f1_weighted", metric_func=metrics.f1_score, args={"average": "weighted"})
roadclf.register_metric(name="accuracy", metric_func=metrics.accuracy_score, args={"normalize": "True"})
roadclf.register_metric(name="AUC", metric_func=metrics.roc_auc_score, args={"multi_class": "ovo"}, proba=True)

In [46]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gae = GAEModel(1,  128, device=device) # pseudo init
gae.load_emb(path="../model_states/gae/embedding.out")
node2vec = Node2VecModel(data=data, device=device) # pseudo init
node2vec.load_emb(path="../model_states/node2vec/embedding.out")
dG = network.line_graph
index = np.arange(len(list(dG.nodes))) # index shifted, since 0 and 1 are reserved for mask pad tokens
nx.set_node_attributes(dG, dict(zip(list(dG.nodes), index)), name="index")
toast = Toast(dG, network.gdf_edges)
toast.load_model("../model_states/toast/model_rw_init_epoch12.pt")

In [47]:
eva = Evaluation()
eva.register_task("Road Type CLF", roadclf)
eva.register_model("GAE", gae)
eva.register_model("Node2Vec", node2vec)
eva.register_model("toast", toast)
res = eva.run()

In [48]:
print(res[0][0])
res[0][1].head()

Road Type CLF


,f1_micro,f1_macro,f1_weighted,accuracy,AUC
GAE,0.715042,0.379847,0.694873,0.715042,0.920333
Node2Vec,0.515218,0.05284,0.353569,0.515218,0.480498
toast,0.737097,0.442612,0.728043,0.737097,0.921767
